In [ ]:
from concurrent.futures import ThreadPoolExecutor
from time import sleep
 
def return_after_5_secs(message):
    sleep(5)
    return message
 
pool = ThreadPoolExecutor(3)
 
future = pool.submit(return_after_5_secs, ("hello"))
print(future.done())
#print(future.result())
sleep(6)
print(future.done())
print(future.result())

In [ ]:
import concurrent

In [ ]:
from logging import init_logger, getLogger

In [ ]:
init_lo

In [ ]:
concurrent.futures.g

In [ ]:
import time
from concurrent.futures import ThreadPoolExecutor
from logging import StreamHandler, Formatter, INFO, getLogger


def init_logger():
    handler = StreamHandler()
    handler.setLevel(INFO)
    handler.setFormatter(Formatter("[%(asctime)s] [%(threadName)s] %(message)s"))
    logger = getLogger()
    logger.addHandler(handler)
    logger.setLevel(INFO)

def task(v):
    getLogger().info("%s start", v)
    time.sleep(1.0)
    getLogger().info("%s end", v)
    return v * 2

def main():
    init_logger()
    getLogger().info("main start")
    with ThreadPoolExecutor(max_workers=2, thread_name_prefix="thread") as executor:
        futures = []
        for i in range(5):
            futures.append(executor.submit(task, i))
        getLogger().info("submit end")
        getLogger().info([f.result() for f in futures])
    getLogger().info("main end")

In [ ]:
main()

In [33]:
from tqdm import tqdm, tqdm_notebook
from concurrent.futures import ProcessPoolExecutor, as_completed
from logging import StreamHandler, Formatter, INFO, getLogger


def init_logger():
    handler = StreamHandler()
    handler.setLevel(INFO)
    handler.setFormatter(Formatter("[%(asctime)s] [%(threadName)s] %(message)s"))
    logger = getLogger()
    if (logger.hasHandlers()):
        logger.handlers.clear()    
    logger.addHandler(handler)
    logger.setLevel(INFO)

def parallel_process(array, function, n_jobs=48, use_kwargs=False, front_num=2, notebook = True):
    """
        A parallel version of the map function with a progress bar. 
        The original function is retrieved from http://danshiebler.com/2016-09-14-parallel-progress-bar/
        Args:
            array (array-like): An array to iterate over.
            function (function): A python function to apply to the elements of array
            n_jobs (int, default=16): The number of cores to use
            use_kwargs (boolean, default=False): Whether to consider the elements of array as dictionaries of 
                keyword arguments to function 
            front_num (int, default=3): The number of iterations to run serially before kicking off the parallel job. 
                Useful for catching bugs
        Returns:
            [function(array[0]), function(array[1]), ...]
    """
    #We run the first few iterations serially to catch bugs
    tqdm_ = tqdm_notebook if notebook else tqdm
    init_logger()
    getLogger().info("calculation start")
    if front_num > 0:
        front = [function(**a) if use_kwargs else function(a) for a in array[:front_num]]
    #If we set n_jobs to 1, just run a list comprehension. This is useful for benchmarking and debugging.
    if n_jobs==1:
        return front + [function(**a) if use_kwargs else function(a) for a in tqdm_(array[front_num:])]
    #Assemble the workers
    with ProcessPoolExecutor(max_workers=n_jobs) as pool:
        #Pass the elements of array into function
        if use_kwargs:
            futures = [pool.submit(function, **a) for a in array[front_num:]]
        else:
            futures = [pool.submit(function, a) for a in array[front_num:]]
        getLogger().info("submit end")
        kwargs = {
            'total': len(futures),
            'unit': 'it',
            'unit_scale': True,
            'leave': True
        }
        #Print out the progress as tasks complete
        getLogger().info("Progress of the calculation")
        for f in tqdm_(as_completed(futures), **kwargs):
            pass
    out = []
    #Get the results from the futures.
    getLogger().info("Progress of the aggregation")
    for i, future in tqdm_(enumerate(futures)):
        try:
            out.append(future.result())
        except Exception as e:
            out.append(e)
    getLogger().info("calculation end")
    return front + out

In [46]:
[{"idx1": i, "t1": j} for i in range(10) for j in range(10)]
# {"idx1": i, "t1": j, "times": np.arange(10) for i in range(10) for j in range(10)}

[{'idx1': 0, 't1': 0},
 {'idx1': 0, 't1': 1},
 {'idx1': 0, 't1': 2},
 {'idx1': 0, 't1': 3},
 {'idx1': 0, 't1': 4},
 {'idx1': 0, 't1': 5},
 {'idx1': 0, 't1': 6},
 {'idx1': 0, 't1': 7},
 {'idx1': 0, 't1': 8},
 {'idx1': 0, 't1': 9},
 {'idx1': 1, 't1': 0},
 {'idx1': 1, 't1': 1},
 {'idx1': 1, 't1': 2},
 {'idx1': 1, 't1': 3},
 {'idx1': 1, 't1': 4},
 {'idx1': 1, 't1': 5},
 {'idx1': 1, 't1': 6},
 {'idx1': 1, 't1': 7},
 {'idx1': 1, 't1': 8},
 {'idx1': 1, 't1': 9},
 {'idx1': 2, 't1': 0},
 {'idx1': 2, 't1': 1},
 {'idx1': 2, 't1': 2},
 {'idx1': 2, 't1': 3},
 {'idx1': 2, 't1': 4},
 {'idx1': 2, 't1': 5},
 {'idx1': 2, 't1': 6},
 {'idx1': 2, 't1': 7},
 {'idx1': 2, 't1': 8},
 {'idx1': 2, 't1': 9},
 {'idx1': 3, 't1': 0},
 {'idx1': 3, 't1': 1},
 {'idx1': 3, 't1': 2},
 {'idx1': 3, 't1': 3},
 {'idx1': 3, 't1': 4},
 {'idx1': 3, 't1': 5},
 {'idx1': 3, 't1': 6},
 {'idx1': 3, 't1': 7},
 {'idx1': 3, 't1': 8},
 {'idx1': 3, 't1': 9},
 {'idx1': 4, 't1': 0},
 {'idx1': 4, 't1': 1},
 {'idx1': 4, 't1': 2},
 {'idx1': 4

In [ ]:
(idx1, t1, times, binarray_csc, window, a) for idx1, t1 in enumerate(times)


In [34]:
def get_big_number(i, x = int(1e+05)):
    return sum([x*x*i for i in range(x)])

In [35]:
list_of_big_number = parallel_process(range(int(1e+04)), get_big_number)

[2018-07-12 17:30:31,022] [MainThread] calculation start
[2018-07-12 17:30:36,054] [MainThread] submit end
[2018-07-12 17:30:36,062] [MainThread] Progress of the calculation


A Jupyter Widget

[2018-07-12 17:30:43,146] [MainThread] Progress of the aggregation


A Jupyter Widget

[2018-07-12 17:30:43,211] [MainThread] calculation end


In [37]:
def get_big_number(i, x = int(1e+05)):
    return sum([x*x*i for i in range(x)])

In [ ]:
arr = []

In [41]:
get_big_number(**arr)

TypeError: get_big_number() argument after ** must be a mapping, not list

In [43]:
get_big_number(**arr[0])

49500000

In [40]:
arr = [{"i":1, "x": 100}]

In [44]:
from scipy.sparse import csc_matrix, lil_matrix
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("../spykesim")
import spykesim.editsim as sp
%load_ext cython
%matplotlib inline

In [18]:

items = [1, 2, 3, 4, 5]
squared = list(map(lambda x: x**2, items))

In [19]:
squared

[1, 4, 9, 16, 25]

In [20]:
a = map(lambda x: x**2, items)

In [21]:
for b in a: 
    print(b)

1
4
9
16
25


In [11]:
from functools import partial

In [48]:
def f(x = 1, k = -3, y = 2, z = -1):
    print("x", x)
    print("k", k)
    print("y", y)
    print("z", z)
    return x + y + z

In [49]:
a = 10

In [50]:
func = partial(f, z = a, k = 1)

In [51]:
func(5, -20)

TypeError: f() got multiple values for argument 'k'

In [31]:
items = enumerate([1,2])
#list(map(lambda x: x[0]+x[1], items))

In [35]:
def plusone(x):
    return x + 1

In [37]:

map(plusone,
    map(
        lambda x: x[0]+x[1], items
    )
   )

In [33]:
for  in map(
    lambda x: x[0]+x[1], items
):
    print(i)

SyntaxError: can't assign to operator (<ipython-input-33-d1ee88fd4d00>, line 1)

In [17]:
map(lambda x: x[0]+x[1], items)